In [1]:
import os
import sys
paths = {
    'helpers': '/home/spadela/mids-w210-capstone'
}
for k,v in paths.items():
    sys.path.insert(0, v)
    
import requests
import pandas as pd
import numpy as np
import datetime
from copy import deepcopy

from bs4 import BeautifulSoup

### Set Up Mongo Connection
from db import dbToolsMongo
db = dbToolsMongo()

import time
import urllib.parse

pd.set_option('display.max_columns', 500)

%reload_ext autoreload
%autoreload 2

In [26]:
file = 'ICO_Reddit_Data_v2.csv'
df1 = pd.read_csv(file,delimiter='|')
df1.head()

,Unnamed: 0,ICO,STARTUP_NAME,HYPE_SCORE,RISK_SCORE,STATUS,ICO_CODE,RISK_CODE,HYPE_CODE,HYPE_CODE_BINARY,ICO_Updated
0,0,NEO,NEO (NEO),VERY HIGH,VERY LOW,POST,NEO,1,5,1,neo
1,1,Cindicator,Cindicator (CND),HIGH,VERY LOW,POST,CND,1,4,1,cindicator
2,2,UnikoinGold,UnikoinGold (UKG),VERY HIGH,VERY LOW,POST,UKG,1,5,1,unikoingold
3,3,Opus,Opus (OPT),VERY HIGH,VERY LOW,POST,OPT,1,5,1,opus
4,4,Shping,Shping (SHPING),MEDIUM,VERY LOW,POST,SHPING,1,3,1,shping


In [27]:
df1['DECRYPTO_RISK_LABEL'] = df1.apply(lambda x: 'HIGH' if x['RISK_CODE'] == 1 else 'LOW', axis=1)
df1['DECRYPTO_HYPE_LABEL'] = df1.apply(lambda x: 'HIGH' if x['HYPE_CODE_BINARY'] == 1 else 'LOW', axis=1)
df1 = df1[['ICO_Updated','HYPE_CODE_BINARY','RISK_CODE','DECRYPTO_RISK_LABEL','DECRYPTO_HYPE_LABEL']]
df1.columns = ['ICO_JOIN_KEY','HYPE_CODE_BINARY','RISK_CODE','RISK SCORE','HYPE SCORE']

In [28]:
db.db['sp_hype_score'].delete_many({})
db.db['sp_hype_score'].insert_many(df1.to_dict('records'))

In [37]:
hype_codes = db.toDF('sp_hype_score')
hype_codes.head()

,HYPE SCORE,HYPE_CODE_BINARY,ICO_JOIN_KEY,RISK SCORE,RISK_CODE,_id
0,HIGH,1,neo,HIGH,1,5b4fde7857c4101afeceb81f
1,HIGH,1,cindicator,HIGH,1,5b4fde7857c4101afeceb820
2,HIGH,1,unikoingold,HIGH,1,5b4fde7857c4101afeceb821
3,HIGH,1,opus,HIGH,1,5b4fde7857c4101afeceb822
4,HIGH,1,shping,HIGH,1,5b4fde7857c4101afeceb823


In [42]:
hype_codes.columns

Index(['HYPE SCORE', 'HYPE_CODE_BINARY', 'ICO_JOIN_KEY', 'RISK SCORE',
       'RISK_CODE', '_id'],
      dtype='object')

In [43]:
hype_codes.groupby(['RISK SCORE','RISK_CODE']).agg({'ICO_JOIN_KEY': 'count'})

ICO_JOIN_KEY
RISK SCORE RISK_CODE              
HIGH       1                     9
LOW        2                   126
           3                   483
           4                   291
           5                    16

In [33]:
df1.columns

Index(['ICO_JOIN_KEY', 'HYPE_CODE_BINARY', 'RISK_CODE', 'RISK SCORE',
       'HYPE SCORE'],
      dtype='object')